# Nettoyage et Importation des données

Dans ce notebook, nous allons effectuer un nettoyage de nos données, que nous allons stocker dans des fichiers .csv, afin de pouvoir les importer dans les notebooks correspondants sans avoir trop de calculs ni de temps d'exécution excessif, ce qui pourrait causer des problèmes.

Nous avons choisi de nous concentrer sur les articles de sport liés à Amazon. Plus précisément, nous utiliserons les données issues du dataset 5-core, qui est beaucoup moins volumineux en mémoire, mais largement suffisant pour nos analyses (voire même excessif pour certaines parties).

Nous allons donc importer ces données :

In [1]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Sports_and_Outdoors_5.json.gz')
print(len(df))
df.head()

2839940


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,vote,image
0,5.0,True,"06 3, 2015",A180LQZBUWVOLF,0000032034,Michelle A,What a spectacular tutu! Very slimming.,Five Stars,1433289600,NaN,NaN,NaN
1,1.0,True,"04 1, 2015",ATMFGKU5SVEYY,0000032034,Crystal R,What the heck? Is this a tutu for nuns? I know...,Is this a tutu for nuns?!,1427846400,NaN,NaN,NaN
2,5.0,True,"01 13, 2015",A1QE70QBJ8U6ZG,0000032034,darla Landreth,Exactly what we were looking for!,Five Stars,1421107200,NaN,NaN,NaN
3,5.0,True,"12 23, 2014",A22CP6Z73MZTYU,0000032034,L. Huynh,I used this skirt for a Halloween costume and ...,I liked that the elastic waist didn't dig in (...,1419292800,NaN,NaN,NaN
4,4.0,True,"12 15, 2014",A22L28G8NRNLLN,0000032034,McKenna,This is thick enough that you can't see throug...,This is thick enough that you can't see throug...,1418601600,NaN,NaN,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2839940 entries, 0 to 2839939
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   reviewerName    object 
 6   reviewText      object 
 7   summary         object 
 8   unixReviewTime  int64  
 9   style           object 
 10  vote            object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 262.7+ MB


On constate que nous disposons de plus de 2 500 000 données, ce qui est encore largement suffisant. Nous pouvons donc envisager de réduire ce nombre si nécessaire.

Dans un premier temps, nous allons effectuer un nettoyage des données en conservant uniquement les achats vérifiés.

In [3]:
# enlever les non verifiés
df = df[df.verified == True]
len(df)

2611805

Nos données sont en quantité relativement similaire.

Nous allons maintenant conserver uniquement les colonnes qui nous intéressent pour la suite du projet, afin d'améliorer la lisibilité.

In [4]:
df = df[['overall', 'reviewerID', 'asin', 'reviewTime']]
df = df.rename(columns={'overall': 'rating', 'reviewerID': 'user', 'asin': 'item', 'reviewTime': 'reviewTime'})
df

,rating,user,item,reviewTime
0,5.0,A180LQZBUWVOLF,0000032034,"06 3, 2015"
1,1.0,ATMFGKU5SVEYY,0000032034,"04 1, 2015"
2,5.0,A1QE70QBJ8U6ZG,0000032034,"01 13, 2015"
3,5.0,A22CP6Z73MZTYU,0000032034,"12 23, 2014"
4,4.0,A22L28G8NRNLLN,0000032034,"12 15, 2014"
...,...,...,...,...
2839935,5.0,A3TN0U64HONOPB,B01HJGAJ9O,"04 5, 2018"
2839936,5.0,AJDQLM8PT3YWT,B01HJGAJ9O,"03 1, 2018"
2839937,5.0,A3QK5ZLRE2KHLL,B01HJHHBHG,"03 17, 2018"
2839938,5.0,A3VDML80KNR9QQ,B01HJHHBHG,"10 28, 2017"


Nous pouvons maintenant analyser le nombre d'utilisateurs et de produits distincts dans notre dataset :

In [5]:
# Afficher stat de notre dataset
print(len(df['user'].unique()))
print(len(df['item'].unique()))

329602
104043


Dans le but d'obtenir les notes de nos achats, nous allons également télécharger les données issues du dataset metadata.

In [1]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df2 = getDF('meta_Sports_and_Outdoors.json.gz')
df2

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Adult Tutu Assorted Colors (Turquoise),[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (","[B071LQWQBQ, B00M14DG0O, B076GTTY9W, B07BSZ4GY...","<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,[],[],NaN
1,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Bububibi Adult Ballet Tutu Cheetah Pink,[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,[],[],NaN
2,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Girls Ballet Tutu Neon Orange,[B01MYHD3LV],,Unknown,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#924,198 in Toys & Games (See Top 100 in Toy...","[B0152HBC52, B07C29WWV1, B01M0BIP9H, B00VXJHFR...",Toys & Games,,,$5.70,0000031860,[],[],NaN
3,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,0000031852,[],[],NaN
4,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[Dance tutu for girls ages 2-8 years. Perfect ...,,Girls Ballet Tutu Neon Blue,"[B009THXHPA, B00BEU1ZBI, B079PYLZTM, B00BEU1ZR...",,Bububibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#393,597 in Toys & Games (See Top 100 in Toy...","[B009THXHPA, B079PYLZTM, B079PQPLT3, B00UP6A3R...",Toys & Games,,,$6.54,0000031895,[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962295,"[Sports & Outdoors, Sports & Fitness, Golf, On...",,[],,Aweoods Inverted Umbrella Windproof Reverse Fo...,[],,eShops ltd,[],"544,488 in Sports & Outdoors (","[B06Y4Z11XM, B07C8GQR5Y, B071JL6JV8, B07C1DPWR...",Sports & Outdoors,,,$49.99,B01HJH3742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,"{'Shipping Weight:': '1.2 pounds (', 'ASIN: ':..."
962296,"[Sports & Outdoors, Outdoor Recreation, Campin...",,[],,JXULE Outdoor Sports Camping Home First Aid Ki...,[],,,[],"1,436,289 in Sports & Outdoors (",[],Sports & Outdoors,,,,B01HJGNPLI,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{'ASIN: ': 'B01HJGNPLI'}
962297,"[Sports & Outdoors, Outdoor Recreation, Access...",,[],,T&amp;s Shell Catcher Beretta A400 Multitarget...,"[B001U9YK8I, B005NH4NQC, B003QYGUME, B007URNR9...",,STS,[],"570,537 in Sports & Outdoors (","[B00VM87O3A, B001U9YK8I, B07GBG321Z, B07CTRPK4...",Sports & Outdoors,,,,B01HJHHBHG,[],[],"{'Shipping Weight:': '1 pounds', 'ASIN: ': 'B0..."
962298,"[Sports & Outdoors, Sports & Fitness]",,[2XU Women's Fitness Compression Tights],,2XU Women's Fitness Compression Tights,"[B01BISC5UA, B00O5XUS6O, B01B2LUO38, B0751MMZ3...",,2XU,"[62% Nylon / 38% Elastane (Invista LYCRA), Mad...","25,733 in Sports & Outdoors (","[B01BISC5UA, B00O5XUS6O, B079P8JG52, B01B2LUO3...",Sports & Outdoors,,,$35.80 - $89.95,B01HJGIZ2W,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{' Product Dimensions: ': '3 x 2 x 12...


Nous souhaitons conserver uniquement les produits appartenant à notre catégorie :

In [2]:
df2 = df2[df2.main_cat == 'Sports & Outdoors']
len(df2)

810640

Nous souhaitons conserver uniquement les colonnes correspondant à l'ID de l'article et à son prix, dans le but de fusionner nos deux datasets. Nous allons donc renommer nos colonnes et convertir la colonne des prix en float en supprimant le symbole dollar ainsi que les valeurs vides, qui indiquent un prix inconnu.

In [3]:
df2 = df2[['asin', 'price']]
df2 = df2.rename(columns={'asin': 'item'})
df2 = df2.dropna(subset=['price'])
df2 = df2[df2['price'] != '']
df2['price'] = df2['price'].str.replace('$', '')
df2['price'] = pd.to_numeric(df2['price'], errors='coerce')
df2 = df2.dropna(subset=['price'])
df2

,item,price
3,0000031852,7.50
6,0000012300,4.00
7,0000031909,6.94
9,0022320113,19.99
13,0692747222,9.97
...,...,...
962282,B01HJDGJ1E,10.99
962284,B01HJDDVYW,14.35
962289,B01HJGB038,78.00
962294,B01HJGDVEE,9.96


Dans le but de faciliter l'exécution du notebook, nous enregistrons nos données dans un fichier .csv, que nous téléchargerons lors de l'exécution.

In [4]:
# Enregirster le dataframe df2 en local
df2.to_csv('df2_temp.csv', index=False)

Nous revenons ainsi à notre dataset de base, que nous allons fusionner avec les notes de nos articles. De plus, nous allons supprimer les transactions pour lesquelles nous ne connaissons pas le prix.

In [6]:
df2 = pd.read_csv('df2_temp.csv')
df = pd.merge(df, df2, on='item')
df = df.dropna(subset=['price'])
df

,rating,user,item,reviewTime,price
0,5.0,A3QIWQM4HVNR0A,0971100764,"07 16, 2016",12.96
1,5.0,A3QIWQM4HVNR0A,0971100764,"07 16, 2016",12.96
2,5.0,A1M9XEAH62IUPW,0971100764,"07 5, 2016",12.96
3,5.0,A2U7YP4MP7BICG,0971100764,"03 25, 2016",12.96
4,2.0,AIFK4MYNSAW4D,0971100764,"02 28, 2016",12.96
...,...,...,...,...,...
1469998,2.0,AW46USVHC6394,B01HJ8YC0Y,"08 11, 2018",18.99
1469999,5.0,A1HCLZ57P6O0YM,B01HJDGJ1E,"09 11, 2018",10.99
1470000,3.0,A2QTOTWE5ZRCZK,B01HJDGJ1E,"08 24, 2018",10.99
1470001,5.0,A2NJSBP40UVZJ8,B01HJDGJ1E,"08 18, 2018",10.99


Voici ainsi les statistiques de nos données actuelles :

In [7]:
# Afficher stat de notre dataset
print(len(df))
print(len(df['user'].unique()))
print(len(df['item'].unique()))

1470003
313220
50082


Nous constatons que nos données incluent des informations sur les dates d'achat. Nous allons donc transformer ces données en format datetime afin de pouvoir analyser les années d'achat.

In [8]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y')
df['reviewTime'].dt.year.value_counts().sort_index()

,count
reviewTime,
2003,4
2005,18
2006,64
2007,509
2008,1101
2009,2547
2010,6318
2011,15512
2012,36330


On remarque que les années avant 2011 ont très peu de données par rapport à celles qui suivent. Nous allons donc les supprimer.

In [9]:
# garder uniquement les reviewTime Supérieur à l'année 2014
df = df[df['reviewTime'].dt.year >= 2011]
len(df)

1459442

Ainsi, voici les statistiques de notre dataset final :

In [11]:
# Afficher stat de notre dataset
df = df[['rating', 'user', 'item', 'price']]
print(f'Nombre de transaction: {len(df)}')
print('Nombre de user:', len(df['user'].unique()))
print('Nombre de item:', len(df['item'].unique()))

Nombre de transaction: 1459442
Nombre de user: 312284
Nombre de item: 50075


Nous enregistrons ainsi nos données en local :

In [12]:
# Enregirster le dataframe df en local
df.to_csv('df.csv', index=False)
df

,rating,user,item,price
0,5.0,A3QIWQM4HVNR0A,0971100764,12.96
1,5.0,A3QIWQM4HVNR0A,0971100764,12.96
2,5.0,A1M9XEAH62IUPW,0971100764,12.96
3,5.0,A2U7YP4MP7BICG,0971100764,12.96
4,2.0,AIFK4MYNSAW4D,0971100764,12.96
...,...,...,...,...
1469998,2.0,AW46USVHC6394,B01HJ8YC0Y,18.99
1469999,5.0,A1HCLZ57P6O0YM,B01HJDGJ1E,10.99
1470000,3.0,A2QTOTWE5ZRCZK,B01HJDGJ1E,10.99
1470001,5.0,A2NJSBP40UVZJ8,B01HJDGJ1E,10.99
